In [1]:
import pandas as pd
import numpy as np

import data.breathe_data as bd
import data.helpers as dh
import models.builders as mb
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import inference.helpers as ih

In [2]:
# Checked that obs indices are correct, see ipynb mentioned above(01.05.2025)
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [2]:
import data.smartcare_O2_FEV1_df as smartcare_O2_FEV1_df
df_sc = smartcare_O2_FEV1_df.create()


** Loading measurements data **

* Processing measures *

FEV1
* Analysing same day duplicates *
* Correcting measurements done before 2am *


INFO:root:* Checking for same day measurements *


Dropped 1062 same day FEV1 duplicates
Dropping 1 entries with FEV1 = 3.45 for user Kings004

O2 Saturation
* Analysing same day duplicates *
* Correcting measurements done before 2am *
Dropped 84 same day O2 Saturation duplicates
IDs with O2 Saturation outside 70-100 % range: 
          UserName  O2 Saturation
3575      Kings005            127
4836   Papworth001            127
4911   Papworth002            127
6257   Papworth010            103
6261   Papworth010            104
8131   Papworth027            127
8265   Papworth028            127
8284   Papworth028            127
10854   leeds01222            127
11065   leeds01320            127
Dropping 10 entries with O2 Saturation outside 70-100 % range

Loaded measurements data with 14451 entries (initially 129841, removed 115390)

** Loading ID map **
Correct ID 101's Patient_ID from ['0HeWh64M_zc5U512xqzAs4'] to 0HeWh64M_zc5U5l2xqzAs4
Correct ID 125's Patient_ID from ['1au5biSTt0bNWgfl0Wltr5'] to 1au5biSTt0bNWgfI0WItr5
Correct ID 2

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():



* Dropping unnecessary columns from patient data *
Columns filtered: ['ID', 'Study Date', 'DOB', 'Age', 'Sex', 'Height', 'Weight', 'Predicted FEV1', 'FEV1 Set As']
Columns dropped: {'GP Letter Sent', 'Date Last PE Stop', 'Hospital', 'Freezer Required', 'Transplant Recipients', 'Age 18 Years', 'Date Last PE Start', 'Unable Sputum Samples', 'Less Exacerbation', 'Genetic Testing', 'Telemetric Measures', 'Unable Informed Consent', 'Informed Consent', 'Date Consent Obtained', 'CFQR Quest Comp', 'Study Email', 'Study Number', 'Sputum Samples', 'Pulmonary Exacerbation', 'Comments', 'Inconvenience Payment', 'Remote Monitoring App User ID'}

* Correcting patient data *
ID 60: Corrected height 60 from 1.63 to 163.0
ID 66: Corrected height for ID 66 from 1.62 to 162.0
Replace Age by calculate age
Drop FEV1 Set As and Predicted FEV1
Compute Calculated Predicted FEV1 using GLI reference equations

* Applying data sanity checks *
Loaded patient data with 147 entries (147 initially)

** Creating Dat

INFO:root:ID 82 - Outlier up for ecFEV1, day 2015-11-20: 2.70 > 2.44 and > 2.08 (mean=1.58,std=0.29), update to 1.84



Created df_O2_FEV1 with 9574 entries (initially 14434, removed 4860)


In [3]:
df_fev1 = bd.load_meas_from_excel(
    "infer_AR_using_fev1_01052025", ["AR", "HFEV1", "HO2Sat"], bypass_sanity_checks=True
).drop(columns=["HO2Sat"])
df_fev1.rename(
    {
        "AR": "AR_fev1",
        "HFEV1": "HFEV1_fev1",
    },
    axis=1,
    inplace=True,
)

In [4]:
df_fev1_o2 = bd.load_meas_from_excel(
    "infer_AR_using_o2sat_fev1_01052025", ["AR", "HFEV1"], bypass_sanity_checks=True
).drop(columns=["IA", "HO2Sat"])
df_fev1_o2.rename(
    {
        "AR": "AR_fev1_o2",
        "HFEV1": "HFEV1_fev1_o2",
    },
    axis=1,
    inplace=True,
)

In [5]:
df_fev1_o2_2d = bd.load_meas_from_excel(
    "infer_AR_using_two_days_model_o2_fev1_01052025",
    ["Airway resistance (%)", "Healthy FEV1 (L)"],
    date_cols=["Day"],
    bypass_sanity_checks=True,
).drop(columns=["Healthy O2 saturation (%)"])
df_fev1_o2_2d.rename(
    {
        "Airway resistance (%)": "AR_fev1_o2_2d",
        "Healthy FEV1 (L)": "HFEV1_fev1_o2_2d",
        "Day": "Date Recorded",
    },
    axis=1,
    inplace=True,
)

In [150]:
df_fev1_o2_fef2575_2d = bd.load_meas_from_excel(
    "infer_AR_using_two_days_model_o2_fev1_fef2575_01052025",
    ["Airway resistance (%)", "Healthy FEV1 (L)"],
    date_cols=["Day"],
    bypass_sanity_checks=True,
).drop(columns=["Healthy O2 saturation (%)"])
df_fev1_o2_fef2575_2d.rename(
    {
        "Airway resistance (%)": "AR_fev1_o2_fef2575_2d",
        "Healthy FEV1 (L)": "HFEV1_fev1_o2_fef2575_2d",
        "Day": "Date Recorded",
    },
    axis=1,
    inplace=True,
)

In [152]:
# join df to df_fev1 on ID and Date Recorded
cols = [
    "ID",
    "Date Recorded",
    "ecFEV1 % Predicted",
    "O2 Saturation",
    "Height",
    "Age",
    "Sex",
]
df_res = pd.merge(df[cols], df_fev1, on=["ID", "Date Recorded"], how="inner")
df_res = pd.merge(df_res, df_fev1_o2, on=["ID", "Date Recorded"], how="inner")
df_res = pd.merge(df_res, df_fev1_o2_2d, on=["ID", "Date Recorded"], how="inner")
df_res = pd.merge(
    df_res, df_fev1_o2_fef2575_2d, on=["ID", "Date Recorded"], how="inner"
)

In [153]:
# Add records count per ID
df_count = (
    df_res.groupby("ID")
    .apply(lambda df: len(df))
    .reset_index()
    .rename(columns={0: "count"})
)
df_res = pd.merge(df_res, df_count, on=["ID"])

In [154]:
# Add max ecFEV1 % pred
df_max_fev = (
    df_res.groupby("ID")
    .apply(lambda df: df["ecFEV1 % Predicted"].max())
    .reset_index()
    .rename(columns={0: "max ecFEV1 % Predicted"})
)
df_res = pd.merge(df_res, df_max_fev, on=["ID"])
df_res["FEV delta"] = df_res["max ecFEV1 % Predicted"] - df_res["ecFEV1 % Predicted"]

In [155]:
# Add avg o2 sat
df_avg_o2 = (
    df_res.groupby("ID")
    .apply(lambda df: df["O2 Saturation"].mean())
    .reset_index()
    .rename(columns={0: "Avg O2 saturation"})
)
df_res = pd.merge(df_res, df_avg_o2, on=["ID"])

In [156]:
# df_res.sort_values(by="FEV delta", ascending=False)[0:5]
# df_res.sort_values(by=["ecFEV1 % Predicted", "O2 Saturation"], ascending=True)[0:5]
indices_max_avg_o2_min_ecfev1 = (
    df_res.groupby("ID")
    .apply(
        lambda df: df.reset_index()
        .sort_values(by=["ecFEV1 % Predicted"], ascending=True)
        .iloc[0]
    )["index"]
    .values
)
indices_over_30_datapoints = df_res[df_res["count"] > 30].index
indices_max_avg_o2_below_94 = df_res[df_res["Avg O2 saturation"] < 94].index
mask = list(
    set(indices_max_avg_o2_min_ecfev1)
    .intersection(indices_max_avg_o2_below_94)
    .intersection(indices_over_30_datapoints)
)
df_res.iloc[mask].sort_values(
    by=["Avg O2 saturation", "ecFEV1 % Predicted"], ascending=[True, True]
).index

Index([3448, 33235, 36519, 35294, 12396], dtype='int64')

# Plot examples of narrowed in posteriors

In [54]:
df_res_last_30[df_res_last_30["ID"] == "101"][0:5].index

Index([1650, 1651, 1652, 1653, 1654], dtype='int64')

In [55]:
# i = 2955
# fev1 and o2: see impact of very low o2
# Impact of min o2 shouldn't get propagated back to hfev1 because it's more due to sudden infection rather than permanent lung damage
# indices = [6311, 35304]
# However, avg o2 is low (or max o2) is relevant. -  Avg o2 sat < 94, min ecfev1, > 30 datapoints
# indices = [35294]
# Unexpectedly small lungs. Below 30% of ecFE1%Predicted (life threatening), expects lower max o2, hence healthy oxygenation must be due to smaller lungs. Ex: o2 = 100, min and max ecfev1.
# → use app
# Exclude o2 from longitudinal model due to calibration issues

# Shared hfev1/ar uncertainty: small healthy lungs or big lungs with disease
# Unexpectedly big lungs with disease: high fev1 % pred delta (benefit of 2nd day)
# indices = [18452, 33198, 2330, 2975, 18430]
# Unexpectedly small lungs: low fev1 % pred but no permanent lung damage (high fef2575%fev1) → show without fef2575 then with

# Super high fev1
# indices = [37113]

indices = [1650, 1651, 1652, 1653, 1654]

for i in indices:
    print(i)
    id = df.loc[i, "ID"]
    (
        _,
        inf_alg,
        HFEV1,
        uFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_point_in_time_model_noise_shared_healthy_vars(
        df_res.iloc[i].Height,
        df_res.iloc[i].Age,
        df_res.iloc[i].Sex,
        ia_prior="uniform",
        ar_prior="uniform",
        ecfev1_noise_model_cpt_suffix="_std_add_mult_ecfev1",
        ar_fef2575_cpt_suffix="_ecfev1_2_days_model_add_mult_noise",
    )

    fig = make_subplots(
        rows=4,
        cols=3,
        shared_xaxes=True,
    )
    first_data_col = 2

    def add_fev1_prct_pred_line(fig, val, y_max, row, col):
        fig.add_shape(
            type="line",
            # opacity=0.1,
            x0=val,
            y0=0,
            x1=val,
            y1=y_max * 1.1,
            line=dict(color="black", width=2, dash="dash"),
            # fillcolor="red",
            # line_width=0,
            row=row,
            col=col,
        )
        return -1

    def add_hists(fig, hfev1_col, ar_col, i, row):
        ih.plot_histogram(
            fig,
            HFEV1,
            df_res.loc[i, hfev1_col],
            HFEV1.a,
            HFEV1.b,
            row,
            first_data_col,
            f" {hfev1_col}",
            "#009e73",
        )
        ih.plot_histogram(
            fig,
            AR,
            df_res.loc[i, ar_col],
            AR.a,
            AR.b,
            row,
            first_data_col + 1,
            f" {ar_col}",
            "#d55e00",
            clean_ticks=True,
        )
        add_fev1_prct_pred_line(
            fig,
            max(100 - df_res.loc[i, "ecFEV1 % Predicted"], 0),
            max(df_res.loc[i, ar_col]),
            row,
            first_data_col + 1,
        )

    ih.plot_histogram(
        fig,
        HFEV1,
        HFEV1.cpt,
        HFEV1.a,
        HFEV1.b,
        1,
        first_data_col,
        HFEV1.name + " prior",
        "#009e73",
    )
    ih.plot_histogram(
        fig,
        AR,
        AR.cpt,
        AR.a,
        AR.b,
        1,
        first_data_col + 1,
        AR.name + " prior",
        "#d55e00",
        clean_ticks=True,
    )
    add_fev1_prct_pred_line(
        fig,
        max(100 - df_res.loc[i, "ecFEV1 % Predicted"], 0),
        max(AR.cpt),
        1,
        first_data_col + 1,
    )

    add_hists(fig, "HFEV1_fev1", "AR_fev1", i, 2)
    add_hists(fig, "HFEV1_fev1_o2", "AR_fev1_o2", i, 3)
    add_hists(fig, "HFEV1_fev1_o2_2d", "AR_fev1_o2_2d", i, 4)

    # Add "prior knowledge" on col 1, row 1
    fig.add_annotation(
        x=0.2,
        y=3.5 / 4,
        text="Prior knowledge",
        xref="paper",
        yref="paper",
        xanchor="center",
        yanchor="middle",
        showarrow=False,
    )
    add_text_box(fig, text="Your text here<br>Second line<br>Third line", row=2, col=1)

    title = f"{id}, {df_res.loc[i, 'Date Recorded']}, i={i}, o2 avg={df_res.loc[i, 'Avg O2 saturation']:.2f}%, ecFEV1%={df_res.loc[i, 'ecFEV1 % Predicted']:.2f}"
    print(title)

    fig.update_layout(
        showlegend=False,
        height=500,
        width=800,
        font=dict(size=11),
        bargap=0.01,
        title=title,
        # margin=dict(l=20, r=20, b=20, t=20),
    )

    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/Narrowing_in/{title}.pdf")

1650
101, 2023-10-13, i=1650, o2 avg=97.46%, ecFEV1%=47.09
1651
101, 2023-10-14, i=1651, o2 avg=97.46%, ecFEV1%=46.81
1652
101, 2023-10-15, i=1652, o2 avg=97.46%, ecFEV1%=44.87
1653
101, 2023-10-16, i=1653, o2 avg=97.46%, ecFEV1%=45.71
1654
101, 2023-10-17, i=1654, o2 avg=97.46%, ecFEV1%=47.09


In [11]:
def add_text_box(fig, text, row, col):
    # Get the subplot's domain coordinates
    x_domain = fig.get_subplot(row, col).xaxis.domain
    y_domain = fig.get_subplot(row, col).yaxis.domain
    # print(x_domain, y_domain)

    # Add rectangle
    fig.add_shape(
        type="rect",
        x0=x_domain[0] - 1,
        x1=x_domain[1] * 25,
        y0=y_domain[0],
        y1=y_domain[1],
        line=dict(color="black", width=0),
        fillcolor="white",
        xref="paper",
        yref="paper",
        row=row,
        col=col,
    )

    # Add text
    fig.add_annotation(
        x=(x_domain[0] + x_domain[1]) / 2,  # center x
        y=(y_domain[0] + y_domain[1]) / 2,  # center y
        text=text,
        showarrow=False,
        font=dict(size=12),
        xref="paper",
        yref="paper",
        xanchor="center",
        yanchor="middle",
        row=row,
        col=col,
    )

# Who can the model help?

In [369]:
# Filter out IDs with less than 30 datapoints
min_datapoints = 10
# min_datapoints = 100
df_res_enough_data = df_res[df_res["count"] >= min_datapoints]
print(df_res_enough_data.ID.nunique())

265


In [434]:
# Get the last 30 days of data for each ID
df_res_min_30_ecfev1 = df_res_enough_data.sort_values(["ID", "ecFEV1 % Predicted"])
df_res_min_30_ecfev1 = df_res_min_30_ecfev1.groupby("ID").head(30)

df_res_last_30 = df_res_enough_data.sort_values(["ID", "Date Recorded"])
df_res_last_30 = df_res_last_30.groupby("ID").tail(30)

df_res_first_30 = df_res_enough_data.sort_values(["ID", "Date Recorded"])
df_res_first_30 = df_res_first_30.groupby("ID").head(30)

In [354]:
# Where AR is underestimated
import models.helpers as mh

AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior={"type": "uniform"})


def get_dumbell_plot_data(df_res, ar_row):

    df_res[f"{ar_row} mean"] = df_res[ar_row].apply(lambda row: AR.get_mean(row))
    df_res[f"{ar_row} std"] = df_res[ar_row].apply(lambda row: AR.get_std(row))
    df_res["AR_ecFEV1%"] = 100 - df_res["ecFEV1 % Predicted"]
    df_res[f"{ar_row} - AR_ecFEV1%"] = df_res[f"{ar_row} mean"] - df_res["AR_ecFEV1%"]
    # Add value low and high
    df_res[f"{ar_row} low"] = df_res[f"{ar_row} mean"] - df_res[f"{ar_row} std"]
    df_res[f"{ar_row} high"] = df_res[f"{ar_row} mean"] + df_res[f"{ar_row} std"]

    # Create DataFrame with means in separate columns
    df_means = (
        df_res.groupby("ID")
        .agg(
            {
                f"{ar_row} std": "mean",
                f"{ar_row} mean": "mean",
                "AR_ecFEV1%": "mean",
                f"{ar_row} - AR_ecFEV1%": "mean",
                f"{ar_row} low": "mean",
                f"{ar_row} high": "mean",
            }
        )
        .rename(
            columns={
                f"{ar_row} - AR_ecFEV1%": f"Mean {ar_row} - AR_ecFEV1%",
                f"{ar_row} mean": f"Mean {ar_row} prediction",
                f"{ar_row} std": f"Mean {ar_row} std",
                "AR_ecFEV1%": "Mean AR_ecFEV1%",
                f"{ar_row} low": f"Mean {ar_row} low",
                f"{ar_row} high": f"Mean {ar_row} high",
            }
        )
    ).reset_index()

    ids_sorted = df_means.sort_values(
        by=f"Mean {ar_row} - AR_ecFEV1%", ascending=False
    )["ID"].values

    # Rename Mean ar row low and high to ar row dist
    df_means_2 = df_means.rename(
        columns={
            f"Mean {ar_row} low": f"Mean {ar_row} dist",
            f"Mean {ar_row} high": f"Mean {ar_row} dist",
        },
    )

    # Melt the dataframe to get four rows per ID for three measures
    df_means_melted = pd.melt(
        df_means_2,
        id_vars=["ID"],
        value_vars=[
            "Mean AR_ecFEV1%",
            f"Mean {ar_row} dist",
            f"Mean {ar_row} prediction",
        ],
        var_name="measure",
        value_name="value",
    )

    return df_means_melted.set_index("ID").loc[ids_sorted].reset_index(), ids_sorted


# Dumbell plot

In [454]:
df_for_dumbell = df_res_last_30
# df_for_dumbell = df_res_first_30

df_ar_fev1_o2, ids_sorted = get_dumbell_plot_data(df_for_dumbell, "AR_fev1_o2")
df_ar_fev1_o2_2d, ids_sorted_2d = get_dumbell_plot_data(df_for_dumbell, "AR_fev1_o2_2d")
df_ar_fev1_o2_fef2575_2d, ids_sorted_fef2575 = get_dumbell_plot_data(
    df_for_dumbell, "AR_fev1_o2_fef2575_2d"
)
# df_ar_fev1_o2_2d = get_dumbell_plot_data('AR_fev1_o2_2d')

In [356]:
df_ar_fev1_o2 = df_ar_fev1_o2.set_index("ID").loc[ids_sorted_fef2575].reset_index()
df_ar_fev1_o2_2d = (
    df_ar_fev1_o2_2d.set_index("ID").loc[ids_sorted_fef2575].reset_index()
)
df_ar_fev1_o2_fef2575_2d = (
    df_ar_fev1_o2_fef2575_2d.set_index("ID").loc[ids_sorted_fef2575].reset_index()
)

In [456]:
# Adding 2nd day massively reduces the AR compared to examples with low ecFEV1

import plotly.graph_objects as go

title = f"Dumbell plot of inferred AR vs ecFEV1%(>={min_datapoints} points, last 30 days avg, with id)"


fig = make_subplots(1, 3, horizontal_spacing=0.1, 
                    column_titles=["Mild CF", "Moderate CF", "Severe CF"])


def plot_dumbell_for_df(fig, df, measures, col):
    # For measure[0]
    mask = df["measure"] == measures[0]
    for id in df[mask]["ID"].unique():
        mask_id = df["ID"] == id
        # Add mask
        mask_final = mask_id & mask
        fig.add_trace(
            go.Scatter(
                x=df[mask_final]["value"],
                y=df[mask_final]["ID"],
                mode="lines",
                name="Airway resistance inferred",
                marker=dict(size=2, color="red"),
                line=dict(width=2),
                # legendgroup="Airway resistance predicted",  # Add legend group
                showlegend=True if id == df[mask]["ID"].unique()[0] else False,  # Only show legend for first trace
            ),
            row=1,
            col=col,
        )
    # For measure[1]
    mask = df["measure"] == measures[1]
    fig.add_trace(
        go.Scatter(
            x=df[mask]["value"],
            y=df[mask]["ID"],
            mode="markers",
            name="1 - ecFEV1%Predicted",
            marker=dict(size=3, color="blue"),
            # legendgroup="1 - ecFEV1%Predicted",  # Add legend group
        ),
        row=1,
        col=col,
    )

# Split dataframe between mild, moderate and severe CF lung disease
# Equivalent to Mean AR_ecFEV1% < 30%, 30 to 60 and > 60%
# Get unique IDs and their corresponding Mean AR_ecFEV1% values
mask_ecfev1 = df_ar_fev1_o2_fef2575_2d["measure"] == "Mean AR_fev1_o2_fef2575_2d prediction"
id_ecfev1_values = df_ar_fev1_o2_fef2575_2d[mask_ecfev1].groupby("ID")["value"].mean()

# Split IDs into three groups based on ecFEV1% values
mild_ids = id_ecfev1_values[id_ecfev1_values < 30].index
moderate_ids = id_ecfev1_values[(id_ecfev1_values >= 30) & (id_ecfev1_values <= 60)].index
severe_ids = id_ecfev1_values[id_ecfev1_values > 60].index

# Create the three dataframes
df_mild = df_ar_fev1_o2_fef2575_2d[df_ar_fev1_o2_fef2575_2d["ID"].isin(mild_ids)]
df_moderate = df_ar_fev1_o2_fef2575_2d[df_ar_fev1_o2_fef2575_2d["ID"].isin(moderate_ids)]
df_severe = df_ar_fev1_o2_fef2575_2d[df_ar_fev1_o2_fef2575_2d["ID"].isin(severe_ids)]

# Print the sizes to verify
print(f"Mild size: {len(df_mild)}")
print(f"Moderate size: {len(df_moderate)}")
print(f"Severe size: {len(df_severe)}")

# Plot the three groups
plot_dumbell_for_df(fig, df_mild, ["Mean AR_fev1_o2_fef2575_2d dist", "Mean AR_ecFEV1%"], 1)
plot_dumbell_for_df(fig, df_moderate, ["Mean AR_fev1_o2_fef2575_2d dist", "Mean AR_ecFEV1%"], 2)
plot_dumbell_for_df(fig, df_severe, ["Mean AR_fev1_o2_fef2575_2d dist", "Mean AR_ecFEV1%"], 3)



# Split the dataframe into 3 roughly equal parts
#
#
# ids = df_ar_fev1_o2_fef2575_2d.ID.unique()
# chunk_size = int(np.ceil(len(ids) // 3))
#
# # Get first chunk
# chunk1_ids = ids[0:chunk_size]
#
# # Get second chunk
# chunk2_ids = ids[chunk_size:2*chunk_size]
#
# # Get third chunk
# chunk3_ids = ids[2*chunk_size:len(ids)]
#
# # Create the three dataframes
# df_ar_fev1_o2_fef2575_2d_chunk1 = df_ar_fev1_o2_fef2575_2d.set_index("ID").loc[chunk1_ids].reset_index()
# df_ar_fev1_o2_fef2575_2d_chunk2 = df_ar_fev1_o2_fef2575_2d.set_index("ID").loc[chunk2_ids].reset_index()
# df_ar_fev1_o2_fef2575_2d_chunk3 = df_ar_fev1_o2_fef2575_2d.set_index("ID").loc[chunk3_ids].reset_index()
#
# Print the sizes to verify
# print(f"Chunk 1 size: {len(df_ar_fev1_o2_fef2575_2d_chunk1)}")
# print(f"Chunk 2 size: {len(df_ar_fev1_o2_fef2575_2d_chunk2)}")
# print(f"Chunk 3 size: {len(df_ar_fev1_o2_fef2575_2d_chunk3)}")
#
# plot_dumbell_for_df(fig, df_ar_fev1_o2_fef2575_2d_chunk1, ["Mean AR_fev1_o2_fef2575_2d dist", "Mean AR_ecFEV1%"], 3)
# plot_dumbell_for_df(fig, df_ar_fev1_o2_fef2575_2d_chunk2, ["Mean AR_fev1_o2_fef2575_2d dist", "Mean AR_ecFEV1%"], 2)
# plot_dumbell_for_df(fig, df_ar_fev1_o2_fef2575_2d_chunk3, ["Mean AR_fev1_o2_fef2575_2d dist", "Mean AR_ecFEV1%"], 1)


# Compare different models
#
#
# plot_dumbell_for_df(fig, df_ar_fev1_o2, ["Mean AR_fev1_o2 dist", "Mean AR_ecFEV1%"], 1)
# plot_dumbell_for_df(fig, df_ar_fev1_o2_2d, ["Mean AR_fev1_o2_2d dist", "Mean AR_ecFEV1%"], 2)


fig.update_layout(
    height=800,
    # height=1800,
    width=1200,
    font=dict(size=10),
    showlegend=True,
    title=title,
    plot_bgcolor="white",
    paper_bgcolor="white",
)
print(min(df_ar_fev1_o2_fef2575_2d["value"]))
fig.update_xaxes(
    range=[-25, 91],
    tickvals=[0, 30, 60, 90],
    title=AR.name,
    gridcolor="#2a3f5f",
    zeroline=True,
    zerolinecolor="#2a3f5f",
    zerolinewidth=2,
)
fig.update_yaxes(
    showticklabels=False,
    showgrid=False,
    gridcolor="#2a3f5f",
    zeroline=True,
    zerolinecolor="#2a3f5f",
    zerolinewidth=2,
)
fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/Dumbell_plot_AR_ecFEV1_by_severity/{title}.pdf")
fig.show()

# On this plot, if a person is very sick, it's best fev1 measurement will contain a lot of inflammatory markers (sputum, airway wall inflammation).
# The sicker the person,the more underestimated the AR pred is because the maximum FEV1 blown is not healthy.
# Let's add the FEF25-75.

# 60% of data falls within red band

# TODO: add vertical lines corresponding to key AR values

# Longitudinal AR profile on the web app

Mild size: 536
Moderate size: 448
Severe size: 76
-23.449175221381445


In [250]:
df_count.set_index("ID").loc[["372", "202", "530", "104", "433", "417"]]

,count
ID,
372,41
202,130
530,205
104,222
433,60
417,20


# Compute breathe avg 

In [462]:
ar_dist_BR = df_res.groupby("ID").apply(lambda df: df['AR_fev1_o2_fef2575_2d'].mean()).mean()

In [463]:
title = f"Comparison of the AR priors learnt on Breathe and Smartcare"

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
ih.plot_histogram(fig, AR, ar_dist_BR, AR.a, AR.b, 1, 1, None)
# name trace to breathe data
fig.data[0].name = "Breathe data"
# ih.plot_histogram(fig, AR, ar_dist_SC, AR.a, AR.b, 2, 1, AR.name)
# fig.data[1].name = "Smartcare data"
fig.update_layout(showlegend=True, font=dict(size=10), title=title)
fig.write_image(dh.get_path_to_main() + f"PlotsCrossStudies/{title}.pdf")
fig.show()